# Build and test the stac container

This notebook is linked to https://eoap.github.io/mastering-app-package/containers/stac/

## Goal

Create a container image and run the stac step in a container.


## Setup the environment

In [2]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

## Build the container

Inspect the container file:

In [3]:
cat ${WORKSPACE}/water-bodies/command-line-tools/stac/Dockerfile

FROM docker.io/python:3.10-slim

RUN pip install --no-cache-dir pystac rio_stac loguru click && \
    python -c "import rio_stac"

ADD app.py /app/app.py

ENTRYPOINT []


Build the container using `podman`:

In [4]:
podman build --format docker -t localhost/stac:latest ${WORKSPACE}/water-bodies/command-line-tools/stac


STEP 1/4: FROM docker.io/python:3.10-slim
STEP 2/4: RUN pip install --no-cache-dir pystac rio_stac loguru click &&     python -c "import rio_stac"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.9/182.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 7.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
--> 40cfe012ada
S

Show the `otsu` help:

In [5]:
podman run --rm -it --env=PYTHONPATH=/app localhost/stac:latest python -m app --help

Usage: app.py [OPTIONS]

  Creates a STAC catalog with the water bodies

Options:
  --input-item TEXT  STAC Item URL  [required]
  --water-body TEXT  Water body geotiff  [required]
  --help             Show this message and exit.


## Test the stac step in the container

Generate the STAC Catalog with detected water bodies:

In [6]:
podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --mount=type=bind,source=/workspace/mastering-app-package/runs/otsu.tif,target=/inputs/otsu.tif,readonly \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/stac:latest \
    python \
    -m \
    app \
    --input-item \
    https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A \
    --water-body \
    /inputs/otsu.tif

2024-04-11 13:49:11.766 | INFO     | __main__:to_stac:31 - Creating a STAC Catalog for /inputs/otsu.tif
2024-04-11 13:49:12.485 | INFO     | __main__:to_stac:62 - Done!


List the outputs:

In [7]:
tree ${RUNTIME}

/workspace/mastering-app-package/runs
├── catalog.json
├── crop_green.tif
├── crop_nir.tif
├── norm_diff.tif
├── otsu.tif
└── S2B_10TFK_20210713_0_L2A
    ├── otsu.tif
    └── S2B_10TFK_20210713_0_L2A.json

1 directory, 7 files
